## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [3]:
load_dotenv(override=True)

True

In [5]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("srinidhiyerraguntala@gmail.com")  # Change to your verified sender
    to_email = To("srinidhiyerraguntala@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [6]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [8]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, a cutting-edge SaaS solution designed to simplify and accelerate SOC 2 compliance and audit preparation.

Maintaining compliance can be a daunting task, especially with increasing regulatory scrutiny and evolving best practices. Our AI-powered platform not only automates routine compliance processes but also provides real-time insights to keep your organization ahead of any potential audit challenges.

Here's how ComplAI can benefit your organization:

1. **Efficiency**: Reduce the time spent on compliance-related tasks, allowing your team to focus on core business activities.
2. **Accuracy**: Leverage AI-driven insights to minimize errors and ensure adherence to industry standards.
3. **Readiness**: Prepare for audits seamlessly with comprehensive documentation and reporting features.

I would love the opportunity to discu

In [9]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Simplify Your SOC2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. I’m [Your Name] from ComplAI, where we help organizations like yours streamline their SOC2 compliance efforts with our AI-driven SaaS solution.

Navigating SOC2 compliance can be challenging and time-consuming. Our platform automates much of the process, allowing you to focus on what truly matters—growing your business while maintaining the trust of your clients.

Key benefits of using ComplAI include:

- **Effortless Audit Preparation**: Save time with automated documentation and streamlined workflows.
- **Real-time Compliance Monitoring**: Stay ahead of regulatory requirements with continuous oversight.
- **Enhanced Security Posture**: Identify and mitigate risks proactively.

I would love to offer you a personalized demo to showcase how ComplAI can specifically address your compliance needs and help you prepare for your next audit with confidence.

Are you availab

In [10]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [11]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Don’t Let SOC2 Compliance Make You Go Bananas! 🍌

Hey [First Name],

Ever felt like preparing for a SOC2 audit is akin to deciphering hieroglyphics while balancing on a unicycle? Well, worry no more! 

At ComplAI, we’ve crafted a solution that turns the chaotic world of compliance into a walk in the park (or a leisurely unicycle ride, if you prefer). Our AI-powered tool ensures you meet all SOC2 requirements without losing your sanity—or your lunch.

Imagine this: no more combing through endless documents, no more late-night panic, and definitely no more “Did I really write that?” moments. With ComplAI, you’ll have more time to focus on what you do best—growing your business, or perfecting your banana bread recipe, or both!

Curious to see how we can lighten your compliance load? Let’s have a quick chat. I promise to keep the unicycles to a minimum.

Best,  
[Your Name]  
[Your Position]  
ComplAI  
P.S. The first demo comes with a virtual high-five! 🖐️


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [12]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [13]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [14]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("srinidhiyerraguntala@gmail.com")  # Change to your verified sender
    to_email = To("srinidhiyerraguntala@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [15]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10625bb00>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### And you can also convert an Agent into a tool

In [16]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11777a840>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [17]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10763f920>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x117779ee0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', description='Write 

## And now it's time for our Sales Manager - our planning agent

In [18]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [37]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."


subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [38]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("srinidhiyerraguntala@gmail.com")  # Change to your verified sender
    to_email = To("srinidhiyerraguntala@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [39]:
tools = [subject_tool, html_tool, send_html_email]

In [40]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x117985120>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x11777ab60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 Function

In [41]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [42]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10763f920>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x117779ee0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', description='Write a 

In [43]:
# Improved instructions thanks to student Guillermo F.
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Wait for ALL THREE drafts before proceeding.
 
2. Evaluate and Select: Compare all three drafts carefully. Pick ONLY the single best email based on effectiveness, engagement, and professionalism.
 
3. Handoff for Sending: Pass ONLY the ONE winning email draft to the 'Email Manager' agent.

CRITICAL:
- DO NOT hand off multiple emails
- DO NOT call the Email Manager until you have selected exactly ONE best email
- You must send exactly 1 email total - no more, no less
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to the name in the email from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)
    print(result)

RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    The cold sales email to Alice has been successfully sent! If you need any further assistance or have more emails to send, feel free to let me know.
- 16 new item(s)
- 5 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.

## 🚀 HARD CHALLENGE SOLUTION: Automated SDR with Email Reply Handling

I've created a complete solution in `sdr_auto_reply.py` that:

1. **Receives email replies** via SendGrid Inbound Parse webhook
2. **Tracks conversation history** in SQLite database
3. **Uses AI agents** to generate contextual follow-up responses
4. **Sends automated replies** to keep the sales conversation going

### Architecture Overview

```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│   Prospect      │     │    SendGrid      │     │  Your Server    │
│   replies to    │────▶│  Inbound Parse   │────▶│  (Flask + AI)   │
│   your email    │     │     Webhook      │     │                 │
└─────────────────┘     └──────────────────┘     └────────┬────────┘
                                                          │
                                                          ▼
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│   Prospect      │     │    SendGrid      │     │   AI Agent      │
│   receives      │◀────│   Send Email     │◀────│   generates     │
│   AI response   │     │      API         │     │   response      │
└─────────────────┘     └──────────────────┘     └─────────────────┘
```


### Setup Instructions

#### Step 1: Install Dependencies
```bash
uv pip install flask
```

#### Step 2: Install and Configure ngrok
ngrok creates a public URL that tunnels to your local server.

1. Download ngrok from: https://ngrok.com/download
2. Sign up for a free account and get your auth token
3. Run: `ngrok config add-authtoken YOUR_TOKEN`

#### Step 3: Start the Server
```bash
# Terminal 1: Start the Flask server
cd 2_openai
python sdr_auto_reply.py

# Terminal 2: Start ngrok tunnel
ngrok http 5000
```

Copy the HTTPS URL from ngrok (e.g., `https://a1b2c3d4.ngrok.io`)

#### Step 4: Configure SendGrid Inbound Parse

1. Go to **SendGrid Dashboard** → **Settings** → **Inbound Parse**
2. Click **"Add Host & URL"**
3. Configure:
   - **Receiving Domain**: Your domain (e.g., `parse.yourdomain.com`) or use a subdomain
   - **Destination URL**: `https://your-ngrok-url.ngrok.io/webhook/email`
   - Check **"POST the raw, full MIME message"** (optional)
4. Click **"Add"**

#### Step 5: Configure DNS (MX Records)

For your domain/subdomain, add an MX record:
- **Host**: `parse` (or your subdomain)
- **Priority**: `10`
- **Value**: `mx.sendgrid.net`

**Note**: DNS propagation can take up to 48 hours.


### Testing Without Full Setup

You can test the system immediately using the simulation endpoint! This lets you see the AI agent in action without configuring SendGrid webhooks.


In [ ]:
# Test the SDR Auto-Reply system without full webhook setup!
# First, run the server in a terminal: python sdr_auto_reply.py

import requests

# Simulate a prospect replying to your cold email
def test_sdr_reply():
    response = requests.post(
        "http://localhost:5000/test/simulate",
        json={
            "from": "ceo@techstartup.com",
            "name": "Sarah Chen",
            "subject": "Re: Streamline Your SOC2 Compliance",
            "body": """Hi there,

Thanks for reaching out. We're actually in the middle of preparing for our first SOC2 audit 
and it's been quite overwhelming. Our team is small and we don't have dedicated compliance staff.

A few questions:
1. How long does it typically take to get SOC2 compliant using your tool?
2. Do you integrate with AWS and GitHub?
3. What's the pricing like for a 20-person startup?

Looking forward to hearing back.

Best,
Sarah"""
        }
    )
    return response.json()

# Uncomment to test (make sure server is running first!)
result = test_sdr_reply()
print(f"Thread ID: {result['thread_id']}")
print(f"\nAI Response:\n{result['response_body']}")


Thread ID: 84d58cb27ee5

AI Response:
Hi Sarah,

Thank you for your response! I completely understand how overwhelming preparing for a SOC2 audit can be, especially with a small team. 

1. With ComplAI, many users typically see progress towards compliance within a few weeks, depending on their current setup. Our tool streamlines the documentation and implementation processes, reducing the workload on your team.

2. Yes, we integrate seamlessly with both AWS and GitHub, making it easier to manage your compliance requirements without disrupting your existing workflows.

3. As for pricing, we offer flexible plans tailored for startups like yours, and I’d be happy to discuss this in more detail during a demo.

Would you be available for a short call next week? This way, we can explore how ComplAI can specifically meet your needs during this crucial time.

Looking forward to your response!

Best,  
The ComplAI Team


In [ ]:
# Continue the conversation - simulate Sarah replying again!

def continue_conversation():
    response = requests.post(
        "http://localhost:5000/test/simulate",
        json={
            "from": "ceo@techstartup.com",
            "name": "Sarah Chen",
            "subject": "Re: Streamline Your SOC2 Compliance",
            "body": """That sounds great! The AWS and GitHub integrations are exactly what we need.

I'd love to see a demo. I'm available next Tuesday or Wednesday afternoon.
Can you send me a calendar invite?

Thanks,
Sarah"""
        }
    )
    return response.json()

# Uncomment to test the follow-up!
# result = continue_conversation()
# print(f"\nAI Follow-up Response:\n{result['response_body']}")


In [47]:
# View all conversations and their history

def view_conversations():
    # List all conversation threads
    conversations = requests.get("http://localhost:5000/conversations").json()
    print("📧 All Conversation Threads:")
    print("-" * 50)
    for conv in conversations:
        print(f"Thread: {conv['thread_id']}")
        print(f"  Prospect: {conv['prospect_name']} <{conv['prospect_email']}>")
        print(f"  Subject: {conv['subject']}")
        print(f"  Messages: {conv['message_count']}")
        print()
    return conversations

def view_thread_history(thread_id):
    # Get full conversation history
    history = requests.get(f"http://localhost:5000/conversations/{thread_id}").json()
    print(f"\n📜 Conversation History for {thread_id}:")
    print("=" * 60)
    for msg in history:
        direction = "📨 INBOUND" if msg['direction'] == 'inbound' else "📤 OUTBOUND"
        print(f"\n{direction} ({msg['timestamp']})")
        print(f"From: {msg['sender']} → To: {msg['recipient']}")
        print("-" * 40)
        print(msg['body'][:500])
        print()

# Uncomment to view conversations (make sure server is running!)
convs = view_conversations()
if convs:
    view_thread_history(convs[0]['thread_id'])


📧 All Conversation Threads:
--------------------------------------------------
Thread: 84d58cb27ee5
  Prospect: Sarah Chen <ceo@techstartup.com>
  Subject: Re: Streamline Your SOC2 Compliance
  Messages: 2


📜 Conversation History for 84d58cb27ee5:

📨 INBOUND (2026-01-08 22:55:37)
From: ceo@techstartup.com → To: srinidhiyerraguntala@gmail.com
----------------------------------------
Hi there,

Thanks for reaching out. We're actually in the middle of preparing for our first SOC2 audit 
and it's been quite overwhelming. Our team is small and we don't have dedicated compliance staff.

A few questions:
1. How long does it typically take to get SOC2 compliant using your tool?
2. Do you integrate with AWS and GitHub?
3. What's the pricing like for a 20-person startup?

Looking forward to hearing back.

Best,
Sarah


📤 OUTBOUND (2026-01-08 22:55:43)
From: srinidhiyerraguntala@gmail.com → To: ceo@techstartup.com
----------------------------------------
Hi Sarah,

Thank you for your response! I

### Key Features of the SDR Auto-Reply System

| Feature | Description |
|---------|-------------|
| **Webhook Receiver** | Flask endpoint that receives POSTs from SendGrid Inbound Parse |
| **Conversation Tracking** | SQLite database maintains thread history for context |
| **Thread Detection** | Automatically groups related emails by subject + sender |
| **AI Response Generation** | Uses OpenAI Agents SDK to generate contextual replies |
| **Email Sending** | Sends responses via SendGrid API |
| **Testing Endpoint** | `/test/simulate` lets you test without full webhook setup |

### Potential Enhancements

- 🗓️ **Calendar Integration**: Connect to Google Calendar or Calendly for scheduling demos
- 🎯 **Lead Scoring**: Track engagement and score leads based on responses
- 🚫 **Opt-out Handling**: Detect unsubscribe requests and stop automated responses
- 📊 **Analytics Dashboard**: Track open rates, response rates, conversion
- 🔄 **CRM Integration**: Sync conversations with Salesforce, HubSpot, etc.
- 🧠 **Sentiment Analysis**: Adjust tone based on prospect sentiment
